In [1]:
import numpy as np
import pickle
import nibabel as nib
import matplotlib.pyplot as plt
import pandas as pd
import os
import itertools
from tqdm import tqdm
from datautility import *

from scipy.io import loadmat, savemat

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
###################### Importing Packages #########################################
# metric = ['ad', 'ak', 'awf', 'eas_De_par', 'eas_De_perp', 'eas_tort', 'FA', 'ias_Da', 'md', 'mk', 'rd', 'rk']
# 12 metrics in total

whichmask = 'old'

metric = ['ak', 'awf', 'eas_De_par', 'eas_De_perp', 'FA', 'ias_Da', 'md', 'mk']
# 8 metric to use, proposed by Alp
if whichmask == 'new':
    mask_name = ['left_rostral','right_rostral','left_middle','right_middle','left_caudal', 'right_caudal', 'corpus_callosum']
# we do not have prefrontal mask for subject space
else:
    mask_name = ['cc_body', 'cc_genu', 'cc_splenium', 'thal']
    
NEW_PATH = './data/117_Stats_Rep.xls'

new_data = pd.read_excel(NEW_PATH, header=0, index_col=0, sheet_name=0)
new_idx_ori = list(new_data.index)

new_idx = [i[3:-4] for i in new_idx_ori]
# new_idx = new_idx[:-60] + new_idx[-50:-12] # filter mask currently

print(new_idx, len(new_idx))

['TBI001', 'TBI002', 'TBI003', 'TBI004', 'TBI005', 'TBI006', 'TBI007', 'TBI008', 'TBI010', 'TBI011', 'TBI012', 'TBI013', 'TBI014', 'TBI015', 'TBI019', 'TBI020', 'TBI021', 'TBI022', 'TBI023', 'TBI024', 'TBI025', 'TBI026', 'TBI027', 'TBI028', 'TBI029', 'TBI031', 'TBI032', 'TBI033', 'TBI035', 'TBI036', 'TBI037', 'TBI038', 'TBI039', 'TBI040', 'TBI041', 'TBI042', 'TBI043', 'TBI044', 'TBI045', 'TBI046', 'TBI047', 'TBI048', 'TBI049', 'TBI050', 'TBI051', 'TBI052', 'TBI053', 'TBI054', 'TBI055', 'TBI056', 'TBI057', 'TBI058', 'TBI059', 'TBI060', 'TBI061', 'TBI062', 'TBI063', 'TBI064', 'TBI065', 'TBI066', 'TBI067', 'TBI068', 'TBI069', 'TBI070', 'TBI071', 'TBI072', 'TBI073', 'TBN001', 'TBN002', 'TBN003', 'TBN004', 'TBN005', 'TBN006', 'TBN007', 'TBN008', 'TBN009', 'TBN010', 'TBN011', 'TBN012', 'TBN013_2', 'TBN014_2', 'TBN018', 'TBN019', 'TBN020', 'TBN021', 'TBN022', 'TBN023', 'TBN024', 'TBN025', 'TBN026', 'TBN027', 'TBN028', 'TBN029', 'TBN030', 'TBN031', 'TBN032', 'TBN033', 'TBN034', 'TBN035', 'TBN0

## Unifying the '.nii' and '.mat'
* Very annoying for a long time, also fix function get_image_subject

In [3]:
current_path = './data/current_cycle_subj_space_metrics_and_masks/metrics_mat/'
old_path = './data/prior_cycle_subj_space_metrics_and_masks/metrics_mat/'

In [ ]:
for filename in os.listdir(old_path): 
        if filename[-3:] == 'nii':
            src = old_path + filename
            dst = old_path + filename[:-3] + 'mat'
            os.rename(src, dst) 

In [ ]:
for filename in os.listdir(current_path): 
        if filename[-3:] == 'nii':
            src = current_path + filename
            dst = current_path + filename[:-3] + 'mat'
            print(dst)
            os.rename(src, dst) 

## Converting extra image and mask from nii to real mat 

In [ ]:
extra_folder = './data/extra/tbss/'

for r, d, f in os.walk(extra_folder):
    for file in f:
        data_path = os.path.join(r, file)
        data = ((nib.load(data_path)).get_fdata()).astype(np.float32)
        data_dict = {}
        data_dict['vol'] = data
                
        m = r[18:]
        name = data_path[-10:-4]
        if m == 'eas_De_par' or m == 'ias_Da' or m == 'awf' or m == 'eas_De_perp':
            PATH = current_path + 'WMTI' + '_' + m + '_' + name + '.mat'
        elif  m == 'ak':
            PATH = current_path + 'DKI' + '_' + m + '_' + name + '.mat'
        elif m == 'fa' or m =='md' or m =='mk':
            PATH = current_path + 'DTI' + '_' + m + '_' + name + '.mat'
        else:
            PATH = None
            print('metric {} is not implemented yet'.format(m))
        
        if PATH is not None:
            savemat(PATH, data_dict, do_compression=True)

In [5]:
extra_mask = './data/extra/mask_nii/'
current_mask = './data/current_cycle_subj_space_metrics_and_masks/mask_mat/'

for r, d, f in os.walk(extra_mask):
    for file in f:
        
        data_path = os.path.join(r, file)
        data = ((nib.load(data_path)).get_fdata()).astype(np.float32)
        data_dict = {}
        data_dict['vol'] = data
        
        PATH = current_mask + file[:-3] + 'mat'
        savemat(PATH, data_dict, do_compression=True)

In [ ]:
for name in tqdm(new_idx):
    data = get_image_subject(name, metric, 'new', shape=(96, 96, 96), verbose=False)

In [ ]:
test = 'TBI001'
print(test[2])

In [ ]:
def show_slice(data, N):
    '''
    data: 3d ndarray
    N: number of slice taken at X axis
    '''
    slices = np.round (np.linspace (0, data.shape[0], N, endpoint=False)).astype(np.int16)
    fig, ax = plt.subplots(1, N, figsize=(15, 4), sharey=True)

    for i,s in enumerate(slices):
        ax[i].imshow(data[s], cmap='gray')
        
def show_negative(index, metric):
    for m in metric:
        PATH = './data/current_cycle_117subj_all_metrics/mat/'+m+'_TBI'+'{:03}'.format(index)+'.mat'
        data = loadmat(PATH)
        print(data['vol'].shape)
        show_slice(data['vol'], N=8)

In [ ]:
metric = ['ad', 'ak', 'awf', 'eas_De_par', 'eas_De_perp', 'eas_tort', 'FA', 'ias_Da', 'md', 'mk', 'rd', 'rk']
# show_negative(1, metric)

In [ ]:
img0 = loadmat('./data/masks/CC_Splenium_mask.mat')['vol']
img1 = loadmat('./data/masks/CC_Genu_mask.mat')['vol']
img2 = loadmat('./data/masks/CC_Body_mask.mat')['vol']
img3 = loadmat('./data/masks/2_R_thal.mat')['vol']
img4 = loadmat('./data/masks/1_L_thal.mat')['vol']

print(img0.shape)

img = img0 + img1*2 + img2*3 + img3*4 + img4*5

show_slice(img, 15)

# 0. View the Masks
* JHU-ICBM-labels-1mm_CortexWM 
* JHU-ICBM-labels-1mm_7ROI
* Bilateral_prefrontalWM

In [ ]:
PATH = './data/masks'
APPEND = '.nii'
FNAME = ['JHU-ICBM-labels-1mm_CortexWM', 'JHU-ICBM-labels-1mm_7ROI', 'Bilateral_prefrontalWM']

for f in FNAME:
    data_file = os.path.join(PATH, f + APPEND)
    data = ((nib.load(data_file)).get_fdata()).astype(np.float32)
    data = np.squeeze(data)
    
    N = 8
    slices = np.round (np.linspace (0, data.shape[0], N, endpoint=False)).astype(np.int16)
    fig, ax = plt.subplots(1, N, figsize=(15, 4), sharey=True)
    
    for i,s in enumerate(slices):
        ax[i].imshow(data[s], cmap='gray')

## 1. Generating Statistics for Regions into DataFrame
* For the old data, 7 regions are extracted: L_Thal, R_Thal, CC_Genu, CC_Body, CC_Splenium, L_PrefrontalWM, R_PrefrontalWM
* For the new data, 5 regions are extracted: L_Thal, R_Thal, CC_Genu, CC_Body, CC_Splenium
* FA, AD, AK, AWF, Depar, Deparp, TORT, DA, MD, MK, RD, RK are extracted from original data
* FA, MD, MK, AWF, AK, Depar, Deperp, DA are the metrics
* Save to STA_PATH = './data/stats_data_fix.xlsx'

In [ ]:
####################### Setting up the Path and Keys ########################################

OLD_PATH = './data/old_65subj_stats JFR.xlsx'
NEW_PATH = './data/117_Stats_Rep.xlsx'

old_data = pd.read_excel(OLD_PATH, header=0, index_col=0, sheet_name=None)
new_data = pd.read_excel(NEW_PATH, header=0, index_col=0, sheet_name=None)

old_keys = ['all_FA_mean', 'all_FA_std',  # FA
            'all_ad_uthr6_mean', 'all_ad_uthr6_std', # AD
            'all_ak_mean', 'all_ak_std',  # AK
            'all_awf_mean', 'all_awf_std', # AWF
            'all_eas_de_par_uthr6_mean', 'all_eas_de_par_uthr6_std', # DEPAR 
            'all_eas_de_perp_uthr6_mean', 'all_eas_de_perp_uthr6_std',  # DEPERP
            'all_eas_tort_uthr9_mean', 'all_eas_tort_uthr9_std', # TORT
            'all_ias_Da_uthr5_mean', 'all_ias_Da_uthr5_std', # DA
            'all_md_uthr5_mean', 'all_md_uthr5_std', # MD
            'all_mk_uthr5_mean', 'all_mk_uthr5_std', # MK
            'all_rd_uthr5_mean', 'all_rd_uthr5_std', # RD
            'all_rk_uthr5_mean', 'all_rk_uthr5_std' # RK
]

new_keys = ['all_FA_mean', 'all_FA_std', # FA
            'all_ad_uthr6_mean', 'all_ad_uthr6_std', # AD
            'all_ak_mean', 'all_ak_std', # AK
            'all_awf_mean', 'all_awf_std', # AWF
            'all_eas_De_par_mean', 'all_eas_De_par_std', # DEPAR
            'all_eas_De_perp_mean', 'all_eas_De_perp_std', # DEPREP
            'all_tort_mean', 'all_tort_std', # TORT
            'all_Da_mean', 'all_Da_std', # DA
            'all_md_mean', 'all_md_std', # MD
            'all_mk_mean', 'all_mk_std', # MK
            'all_rd_mean', 'all_rd_std', # RD
            'all_rk_mean', 'all_rk_std', # RK
]

mask_keys = ['L_Thal', 'R_Thal', 'CC_Genu', 'CC_Body', 'CC_Splenium']

####################### Rescheduling the columns and row label ##############################

Index_column = ['-'.join(i) for i in list(itertools.product(new_keys, mask_keys))]

data_TBI_index = np.arange(start=1, stop=74)
data_TBN_index = np.arange(start=1, stop=54)

data_TBI_index = np.delete(data_TBI_index, [8,15,16,17,29,33])
data_TBN_index = np.delete(data_TBN_index, [14, 15, 16])

Index_TBI = ['TBI-' + '{:03}'.format(i) for i in data_TBI_index]
Index_TBN = ['TBN-' + '{:03}'.format(i) for i in data_TBN_index]

Index_row = Index_TBI + Index_TBN + list (old_data['all_FA_mean'].index)

data_list = [] # Final Data List

####################### Combining old data with the news ####################################

for okey, nkey in zip(old_keys, new_keys):
    
    old_temp = old_data[okey].iloc[:, 0:5].values
    new_temp = new_data[nkey].values
    
    temp = np.vstack([new_temp, old_temp]) # new on top of old
    data_list.append(temp)

data = np.hstack(data_list)
print('in total {} subjects with {} of features each'.format(*data.shape))

data_dic = pd.DataFrame(data=data, index=Index_row, columns=Index_column)

####################### Save dataframe to Excel #############################################

STA_PATH = './data/stats_data.xlsx'
if not os.path.isfile(STA_PATH):
    data_dic.to_excel(STA_PATH)
else:
    print('file {} exists'.format(STA_PATH))
    
print(data_dic.head())

## 2. Combining Data with the Ground Truth
* In total 154 subs have both data and at least one label
* first 120 columns are features, the rest are labels

In [ ]:
####################### Setting up the Path and Keys ########################################

DATA_PATH = './data/stats_data.xlsx'
LABEL_PATH = './data/merged_np.xlsx'

data_df = pd.read_excel(DATA_PATH, index_col=0)
label_df = pd.read_excel(LABEL_PATH,  sheet_name=1, index_col=1)

sub_data = np.array (list (data_df.index))
sub_label = np.array (list (label_df.index))

haveSubject = (sub_data[None,:] == sub_label[:,None]).astype(np.int16)

sub_index = []

####################### Finding the subjects shared by data and label #######################

for i in range (haveSubject.shape[1]):
    if np.sum(haveSubject[:,i]):
        data_idx = i
        label_idx = np.argmax(haveSubject[:,i])
        sub_index.append((data_idx, label_idx))

sub_index_data = [i[0] for i in sub_index]
sub_index_label = [i[1] for i in sub_index]        

data_df = data_df.iloc[sub_index_data, :]
label_df = label_df.iloc[sub_index_label,:]

whole_data = pd.concat([data_df, label_df], axis=1)

####################### Save dataframe to Excel #############################################

WDTA_PATH = './data/stats_data_label.xlsx'

if not os.path.isfile(WDTA_PATH):
    whole_data.to_excel(WDTA_PATH)
else:
    print('file {} exists'.format(WDTA_PATH))
    
print(whole_data.head())

In [ ]:
data_flitered = whole_data [whole_data['T1 Letter Number'].notnull()]
X = data_flitered.values [:,0:120]
Y = data_flitered.values [:,121:122] # T1 Letter Number

In [ ]:
from sklearn.model_selection import train_test_split

Xtr, Xts, Ytr, Yts = train_test_split(X, Y.ravel(), test_size=0.333)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV, KFold

regr = Lasso()
cv = KFold(n_splits=5, shuffle=True)
parameters = { 'alpha':np.arange(100)/50}

regr_grid = GridSearchCV(regr, parameters, cv = cv, scoring= 'r2', n_jobs=-1, verbose=10)
regr_grid.fit(X, Y)
print(regr_grid.best_score_)

In [ ]:
model = RandomForestRegressor()

parameters = { 'min_samples_leaf':[1,2,4,8,16],'max_depth':[2,4,8,16],'n_estimators':[500]}
model_grid = GridSearchCV(model, parameters, cv = cv, scoring= 'r2', n_jobs=-1, verbose=10)

model_grid.fit(X, Y)
print(model_grid.best_score_)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

Ypred = regr.predict(Xts)
print(mean_squared_error(Yts, Ypred), r2_score(Yts, Ypred))

In [ ]:
X = data_flitered.values [:,idx_sel]
Y = data_flitered.values [:,121:122] # T1 Letter Number

Xtr, Xts, Ytr, Yts = train_test_split(X, Y.ravel(), test_size=0.333)

In [ ]:

model = RandomForestRegressor()

parameters = { 'min_samples_leaf':[5,10],'max_depth':[10],'n_estimators':[500]}
model_grid = GridSearchCV(model, parameters, cv = 5, scoring= 'r2', n_jobs=-1, verbose=10)

model_grid.fit(X, Y)

In [ ]:
print(model_grid.best_score_)

In [ ]:
regr = RandomForestRegressor(max_depth=10,min_samples_leaf=5, random_state=0, n_estimators=500)
regr.fit(Xtr, Ytr)

Ytrhat = regr.predict(Xtr)
Ypred = regr.predict(Xts)
print(mean_squared_error(Ytr, Ytrhat), r2_score(Ytr, Ytrhat))
print(mean_squared_error(Yts, Ypred), r2_score(Yts, Ypred))

In [ ]:
STA_PATH = './data/stats_data.p'
stat = pickle.load(open(STA_PATH, "rb"))

print(stat.shape)